In [1]:
import pandas as pd
import numpy as np

def process_hr_data():
    # 1. Chargement des données
    print("Chargement des fichiers...")
    general_data = pd.read_csv('data/general_data.csv')
    manager_survey = pd.read_csv('data/manager_survey_data.csv')
    employee_survey = pd.read_csv('data/employee_survey_data.csv')
    in_time = pd.read_csv('data/in_time.csv')
    out_time = pd.read_csv('data/out_time.csv')
    # 2. Fusion des données principales
    # On utilise EmployeeID comme clé de jointure
    df_main = general_data.merge(manager_survey, on='EmployeeID', how='left')
    df_main = df_main.merge(employee_survey, on='EmployeeID', how='left')

    # 3. Traitement des fichiers Temps (Badgeuse)
    # Renommer la première colonne vide ou index en 'EmployeeID'
    in_time.rename(columns={in_time.columns[0]: 'EmployeeID'}, inplace=True)
    out_time.rename(columns={out_time.columns[0]: 'EmployeeID'}, inplace=True)

    # Définir l'index pour faciliter les calculs matriciels
    in_time.set_index('EmployeeID', inplace=True)
    out_time.set_index('EmployeeID', inplace=True)

    print("Traitement des horaires (conversion et calcul)...")
    # Conversion en datetime (ignorer les erreurs pour les jours fériés/absences)
    processed_in = in_time.apply(pd.to_datetime, errors='coerce')
    processed_out = out_time.apply(pd.to_datetime, errors='coerce')

    # Calcul de la durée de travail journalière
    duration = processed_out - processed_in
    # Conversion en heures
    duration_hours = duration.apply(lambda x: x.dt.total_seconds() / 3600)

    # Création des nouvelles métriques
    # Moyenne des heures travaillées par jour (en ignorant les NaN/absences)
    mean_hours = duration_hours.mean(axis=1)
    # Nombre total de jours travaillés dans l'année
    working_days = duration_hours.count(axis=1)

    # Création du DataFrame des features temporelles
    time_features = pd.DataFrame({
        'EmployeeID': in_time.index,
        'AverageWorkingHours': mean_hours.values,
        'TotalWorkingDays': working_days.values
    })

    # 4. Fusion finale
    final_df = df_main.merge(time_features, on='EmployeeID', how='left')

    # 5. Nettoyage
    # Suppression des colonnes qui n'ont qu'une seule valeur (inutiles pour le modèle)
    # Exemple: 'Over18' est 'Y' pour tout le monde, 'StandardHours' est 8 pour tout le monde.
    nunique = final_df.apply(pd.Series.nunique)
    cols_to_drop = nunique[nunique == 1].index
    final_df.drop(columns=cols_to_drop, inplace=True)

    print(f"Colonnes supprimées (valeur constante) : {list(cols_to_drop)}")
    print(f"Taille finale du dataset : {final_df.shape}")

    # 6. Export
    final_df.to_csv('processed_hr_data.csv', index=False)
    print("Fichier 'processed_hr_data.csv' généré avec succès.")

    return final_df

# Exécution
if __name__ == "__main__":
    df = process_hr_data()
    print(df.head())

Chargement des fichiers...
Traitement des horaires (conversion et calcul)...
Colonnes supprimées (valeur constante) : ['EmployeeCount', 'Over18', 'StandardHours']
Taille finale du dataset : (4410, 28)
Fichier 'processed_hr_data.csv' généré avec succès.
   Age Attrition     BusinessTravel              Department  DistanceFromHome  \
0   51        No      Travel_Rarely                   Sales                 6   
1   31       Yes  Travel_Frequently  Research & Development                10   
2   32        No  Travel_Frequently  Research & Development                17   
3   38        No         Non-Travel  Research & Development                 2   
4   32        No      Travel_Rarely  Research & Development                10   

   Education EducationField  EmployeeID  Gender  JobLevel  ... YearsAtCompany  \
0          2  Life Sciences           1  Female         1  ...              1   
1          1  Life Sciences           2  Female         1  ...              5   
2          4     

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_hr_data():
    # 1. Chargement des données traitées
    df = pd.read_csv('processed_hr_data.csv')
    
    # Configuration du style
    sns.set(style="whitegrid")
    
    # --- GRAPHIQUE 1 : Distribution de la Cible (Attrition) ---
    plt.figure(figsize=(6, 4))
    ax = sns.countplot(x='Attrition', data=df, palette='viridis')
    plt.title('Répartition des Départs (Attrition)', fontsize=14)
    # Ajout des pourcentages
    total = len(df)
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + p.get_width()/2
        y = p.get_height()
        ax.annotate(percentage, (x, y), ha='center', va='bottom')
    plt.savefig('analyse_1_attrition_globale.png')
    plt.close()

    # --- GRAPHIQUE 2 : Taux de départ par Métier (JobRole) ---
    # On calcule le % de départ pour chaque métier pour voir les plus touchés
    role_attrition = df.groupby('JobRole')['Attrition'].apply(lambda x: (x == 'Yes').mean() * 100).sort_values(ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x=role_attrition.index, y=role_attrition.values, palette='magma')
    plt.xticks(rotation=45, ha='right')
    plt.title('Taux de Départ par Métier (%)', fontsize=14)
    plt.ylabel('Pourcentage de départ')
    plt.tight_layout()
    plt.savefig('analyse_2_attrition_metier.png')
    plt.close()

    # --- GRAPHIQUE 3 : Impact des Heures de Travail (La nouvelle feature) ---
    plt.figure(figsize=(8, 6))
    sns.boxplot(x='Attrition', y='AverageWorkingHours', data=df, palette='coolwarm')
    plt.title('Heures de Travail Moyennes : Ceux qui partent vs Ceux qui restent', fontsize=14)
    plt.ylabel('Heures moyennes par jour')
    plt.savefig('analyse_3_working_hours.png')
    plt.close()

    # --- GRAPHIQUE 4 : Impact de l'Ancienneté (YearsAtCompany) ---
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df[df['Attrition'] == 'Yes']['YearsAtCompany'], label='Départ (Yes)', shade=True, color='red')
    sns.kdeplot(data=df[df['Attrition'] == 'No']['YearsAtCompany'], label='Reste (No)', shade=True, color='blue')
    plt.title("Distribution de l'Ancienneté selon le Départ", fontsize=14)
    plt.legend()
    plt.savefig('analyse_4_anciennete.png')
    plt.close()

    # --- GRAPHIQUE 5 : Satisfaction vs Départ ---
    # Comparaison JobSatisfaction
    plt.figure(figsize=(8, 6))
    sns.countplot(x='JobSatisfaction', hue='Attrition', data=df, palette='Set2')
    plt.title('Satisfaction au Travail et Départ', fontsize=14)
    plt.savefig('analyse_5_satisfaction.png')
    plt.close()

    # --- GRAPHIQUE 6 : Matrice de Corrélation ---
    # Sélection des variables numériques clés
    cols = ['Age', 'MonthlyIncome', 'AverageWorkingHours', 'YearsAtCompany', 
            'DistanceFromHome', 'NumCompaniesWorked', 'TrainingTimesLastYear']
    corr = df[cols].corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title('Matrice de Corrélation des Variables Clés', fontsize=14)
    plt.tight_layout()
    plt.savefig('analyse_6_correlation.png')
    plt.close()

    print("6 graphiques d'analyse ont été générés (fichiers PNG).")

if __name__ == "__main__":
    analyze_hr_data()

C:\Users\Fire\AppData\Local\Temp\ipykernel_10084\373208364.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.countplot(x='Attrition', data=df, palette='viridis')
C:\Users\Fire\AppData\Local\Temp\ipykernel_10084\373208364.py:31: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=role_attrition.index, y=role_attrition.values, palette='magma')
C:\Users\Fire\AppData\Local\Temp\ipykernel_10084\373208364.py:41: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Attrition', y='AverageWorkingHours', data=df, palette='coolwarm')
C:\Users\Fire\AppData\Loca

6 graphiques d'analyse ont été générés (fichiers PNG).
